In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [1]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal
import pickle

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from tslearn.metrics import dtw_path
from fastdtw import fastdtw

2024-06-29 00:30:39.603472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Apple Watchのモーションデータの読み込み
a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
# a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.csv")
# b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
# c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
# d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
# e_okeda_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [32]:
#Tobiiのモーションデータの読み込み
a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okeda_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
# a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.csv")
# b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
# c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
# d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
# e_okeda_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:109: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:109: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:109: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:109: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)


In [5]:
#a_yuumaくんの教師データ読み込み
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross(new)")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross(new)")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [ ]:
#e_okedaくんの教師データ読み込み
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 18:34:55')
circle_end_time = pd.Timestamp('2024-06-04 18:34:59')
cross_start_time = pd.Timestamp('2024-06-04 18:45:10')
cross_end_time = pd.Timestamp('2024-06-04 18:45:14')
tri_start_time = pd.Timestamp('2024-06-04 19:06:48')
tri_end_time = pd.Timestamp('2024-06-04 19:06:52')
check_start_time = pd.Timestamp('2024-06-04 19:21:52')
check_end_time = pd.Timestamp('2024-06-04 19:21:55')

gestures = [
    (circle_start_time, circle_end_time, "Circle"),
    (cross_start_time, cross_end_time, "Cross"),
    (tri_start_time, tri_end_time, "Triangle"),
    (check_start_time, check_end_time, "Check")
]

data = b_sakamoto_motion_data[
    (b_sakamoto_motion_data['Timestamp'] >= (circle_start_time - pd.Timedelta(seconds=5))) &
    (b_sakamoto_motion_data['Timestamp'] <= (circle_end_time + pd.Timedelta(seconds=5)))
]

In [ ]:
Th = 170

In [ ]:
segx, segy, segz = [], [], []
for i in range(len(b_sakamoto_circle)):
    segx.append(mr.spring_ogawa(data['GyroX'].reset_index(drop = True), b_sakamoto_circle[i]['GyroX'], Th))
    segy.append(mr.spring_ogawa(data['GyroY'].reset_index(drop = True), b_sakamoto_circle[i]['GyroY'], Th))
    segz.append(mr.spring_ogawa(data['GyroZ'].reset_index(drop = True), b_sakamoto_circle[i]['GyroZ'], Th))

In [7]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/a_yuuma_acc_circle_segments4.15.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['a_yuuma_circle_segx']
segy = data['a_yuuma_circle_segy']
segz = data['a_yuuma_circle_segz']

In [13]:
filtered_segx = filter_seg_by_elapsed_time(segx, a_yuuma_motion_data, 2, 4)
filtered_segy = filter_seg_by_elapsed_time(segy, a_yuuma_motion_data, 2, 4)
filtered_segz = filter_seg_by_elapsed_time(segz, a_yuuma_motion_data, 2, 4)

In [15]:
filtered_segz

[[(3264, 3388),
  (8759, 8872),
  (9809, 9912),
  (12958, 13060),
  (13452, 13557),
  (14904, 15012),
  (15263, 15418),
  (15528, 15640),
  (16395, 16503),
  (20881, 20991),
  (21865, 22032),
  (22269, 22381),
  (23650, 23770),
  (25061, 25169),
  (25967, 26091),
  (26546, 26651),
  (28286, 28394),
  (28717, 28822),
  (29444, 29560),
  (31993, 32094),
  (34745, 34921),
  (34962, 35065),
  (35922, 36023),
  (36299, 36403),
  (37339, 37453),
  (37492, 37601),
  (39387, 39511),
  (73248, 73364),
  (117652, 117766)],
 [(8759, 8871),
  (9806, 9912),
  (13451, 13557),
  (14903, 15012),
  (18742, 18846),
  (20877, 20991),
  (21393, 21498),
  (21803, 21908),
  (23649, 23770),
  (25060, 25171),
  (25964, 26082),
  (26544, 26651),
  (27466, 27574),
  (28283, 28394),
  (28717, 28823),
  (28876, 28998),
  (29441, 29560),
  (31034, 31145),
  (33861, 33966),
  (34961, 35065),
  (35190, 35292),
  (35921, 36037),
  (36358, 36485),
  (37339, 37453),
  (37488, 37620),
  (39392, 39511),
  (73247, 73364)]

In [ ]:
segx = filter_seg_by_elapsed_time(segx, data.reset_index(drop = True), 2, 4)
segy = filter_seg_by_elapsed_time(segy, data.reset_index(drop = True), 2, 4)
segz = filter_seg_by_elapsed_time(segz, data.reset_index(drop = True), 2, 4)

In [17]:
# 各教師データの結果ごとにオーバーラップを検出
overlap = []
for i in range(len(filtered_segx)):
    overlap.append(mr.combine_and_find_overlapping_filtered_segments(filtered_segx[i], filtered_segy[i], filtered_segz[i]))

In [19]:
filtered_overlap = mr.filter_overlap_by_elapsed_time(overlap, a_yuuma_motion_data, 2, 4)

In [24]:
filtered_overlap

[[(9808, 9935), (23632, 23770)],
 [(24806, 24997), (25363, 25517), (38640, 38832), (117632, 117766)],
 [(23632, 23769), (38641, 38832)]]

In [ ]:
data['Timestamp'].reset_index(drop = True)[520]

In [23]:
a_yuuma_motion_data['Timestamp'][23632]

Timestamp('2024-06-04 16:05:08.295982848')

In [26]:
a_yuuma_combine_overlap = mr.combine_all_overlaps(filtered_overlap)

In [27]:
a_yuuma_combine_overlap

[(9808, 9935),
 (23632, 23770),
 (24806, 24997),
 (25363, 25517),
 (38640, 38832),
 (117632, 117766)]

In [28]:
a_yuuma_results =  mr.extract_timestamp_from_overlap(a_yuuma_motion_data, a_yuuma_combine_overlap)


In [29]:
a_yuuma_results

[[Timestamp('2024-06-04 16:00:31.973660928'),
  Timestamp('2024-06-04 16:00:34.512285952')],
 [Timestamp('2024-06-04 16:05:08.295982848'),
  Timestamp('2024-06-04 16:05:11.054481920')],
 [Timestamp('2024-06-04 16:05:31.762555136'),
  Timestamp('2024-06-04 16:05:35.580347904')],
 [Timestamp('2024-06-04 16:05:42.896267776'),
  Timestamp('2024-06-04 16:05:45.974630144')],
 [Timestamp('2024-06-04 16:10:08.277480960'),
  Timestamp('2024-06-04 16:10:12.115343104')],
 [Timestamp('2024-06-04 16:46:30.329495040'),
  Timestamp('2024-06-04 16:46:33.008007168')]]

In [30]:
a_yuuma_filtered_results = mr.filter_segments_by_time_length(a_yuuma_results, 2, 4)

In [33]:
a_yuuma_extracted_eye_data = mr.extract_eye_data_within_intervals(a_yuuma_filtered_results, a_yuuma_eye_data)

In [34]:
a_yuuma_filtered_extracted_eye_data, a_yuuma_filtered_filtered_results = filter_by_std_gaze(a_yuuma_extracted_eye_data, a_yuuma_filtered_results, 20)

20.197152747937444 0.047566093699622516 0.04647790804788797
28.980274791533372 0.060176911092773115 0.042423431139652044
37.13098854623689 0.05964751808281649 0.1206253286067144
37.62660385940788 0.04583107958536722 0.09941188444420056
38.44841733956389 0.06272030126746692 0.09193915701102873
41.62333612604336 0.03637718024289735 0.1022603569784091


In [ ]:
# Combine all segments from multiple lists into one list
combined_segments = []
for segments in filtered_overlap:
    combined_segments.extend(segments)

# Sort combined segments
combined_segments.sort()

In [ ]:
combined_segments

In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/gyro_tri_segments25.pkl', 'rb') as f:
    data = pickle.load(f)

# 変数を復元する
a_yuuma_tri_segx = data['a_yuuma_tri_segx']
a_yuuma_tri_segy = data['a_yuuma_tri_segy']
a_yuuma_tri_segz = data['a_yuuma_tri_segz']
b_sakamoto_tri_segx = data['b_sakamoto_tri_segx']
b_sakamoto_tri_segy = data['b_sakamoto_tri_segy']
b_sakamoto_tri_segz = data['b_sakamoto_tri_segz']
c_watabe_tri_segx = data['c_watabe_tri_segx']
c_watabe_tri_segy = data['c_watabe_tri_segy']
c_watabe_tri_segz = data['c_watabe_tri_segz']
d_nakazawa_tri_segx = data['d_nakazawa_tri_segx']
d_nakazawa_tri_segy = data['d_nakazawa_tri_segy']
d_nakazawa_tri_segz = data['d_nakazawa_tri_segz']
a_yuuma_tri_overlap = mr.overlap_from_three_segments(a_yuuma_tri_segx, a_yuuma_tri_segy, a_yuuma_tri_segz)
b_sakamoto_tri_overlap = mr.overlap_from_three_segments(b_sakamoto_tri_segx, b_sakamoto_tri_segy, b_sakamoto_tri_segz)
c_watabe_tri_overlap = mr.overlap_from_three_segments(c_watabe_tri_segx, c_watabe_tri_segy, c_watabe_tri_segz)
d_nakazawa_tri_overlap = mr.overlap_from_three_segments(d_nakazawa_tri_segx, d_nakazawa_tri_segy, d_nakazawa_tri_segz)
# 経過時間のフィルタリングを適用
a_yuuma_filtered_overlap = mr.filter_overlap_by_elapsed_time(a_yuuma_tri_overlap, a_yuuma_motion_data, min_time, max_time)
b_sakamoto_filtered_overlap = mr.filter_overlap_by_elapsed_time(b_sakamoto_tri_overlap, b_sakamoto_motion_data, min_time, max_time)
c_watabe_filtered_overlap = mr.filter_overlap_by_elapsed_time(c_watabe_tri_overlap, c_watabe_motion_data, min_time, max_time)
d_nakazawa_filtered_overlap = mr.filter_overlap_by_elapsed_time(d_nakazawa_tri_overlap, d_nakazawa_motion_data, min_time, max_time)
# オーバーラップを統合
a_yuuma_combine_overlap = mr.combine_all_overlaps(a_yuuma_filtered_overlap)
b_sakamoto_combine_overlap = mr.combine_all_overlaps(b_sakamoto_filtered_overlap)
c_watabe_combine_overlap = mr.combine_all_overlaps(c_watabe_filtered_overlap)
d_nakazawa_combine_overlap = mr.combine_all_overlaps(d_nakazawa_filtered_overlap)
# 統合された組からタイムスタンプの組を抽出する
a_yuuma_results =  mr.extract_timestamp_from_overlap(a_yuuma_motion_data, a_yuuma_combine_overlap)
b_sakamoto_results =  mr.extract_timestamp_from_overlap(b_sakamoto_motion_data, b_sakamoto_combine_overlap)
c_watabe_results =  mr.extract_timestamp_from_overlap(c_watabe_motion_data, c_watabe_combine_overlap)
d_nakazawa_results =  mr.extract_timestamp_from_overlap(d_nakazawa_motion_data, d_nakazawa_combine_overlap)
# 各組の時間の長さが2秒以下と5秒以上のものを弾く
a_yuuma_filtered_results = mr.filter_segments_by_time_length(a_yuuma_results, min_time, max_time)
b_sakamoto_filtered_results = mr.filter_segments_by_time_length(b_sakamoto_results, min_time, max_time)
c_watabe_filtered_results = mr.filter_segments_by_time_length(c_watabe_results, min_time, max_time)
d_nakazawa_filtered_results = mr.filter_segments_by_time_length(d_nakazawa_results, min_time, max_time)

In [ ]:
# 区間リストの定義
intervals = [
    (d_nakazawa_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, d_nakazawa_eye_data['Timestamp'].max(), 'Desk work')
]
# 結果をカウントする辞書
interval_counts = {label: 0 for _, _, label in intervals}
a = 0
# 各フィルタされた結果の区間をカウント
for start, end in d_nakazawa_filtered_results:
    for interval_start, interval_end, label in intervals:
        if (interval_start <= start <= interval_end) or (interval_start <= end <= interval_end):
            interval_counts[label] += 1
            a+=1
            break

# 結果を表示
print(interval_counts)
print(a)

In [ ]:
# 結果をカウントする辞書
interval_counts = {label: 0 for _, _, label in intervals}

a = 0
# 各フィルタされた結果の区間をカウント
for start, end in d_nakazawa_filtered_results:
    for interval_start, interval_end, label in intervals:
        if (interval_start <= start <= interval_end) or (interval_start <= end <= interval_end):
            interval_counts[label] += 1
            a+=1
            break

# 結果を表示
print(interval_counts)
print(a)